In [51]:
import sys
sys.executable

'/Users/sitewang/miniconda3/envs/gtcsummary/bin/python'

In [52]:
import os
import PyPDF2

In [53]:
def extract_text_from_first_page(pdf_path):
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        first_page = pdf_reader.pages[0]
        text = first_page.extract_text()
        return text.strip() if text else 'Text Not Found'

def extract_texts_from_directory(directory_path):
    texts = {}
    for filename in os.listdir(directory_path):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(directory_path, filename)
            text = extract_text_from_first_page(pdf_path)
            texts[filename] = text
    return texts

# Specify the directory containing the PDFs
pdf_directory = '../talks/'
texts = extract_texts_from_directory(pdf_directory)

for filename, title in texts.items():
    print(f'Filename: {filename}, Title: {title}')
    break

FloatObject (b'0.00-10') invalid; use 0.0 instead
FloatObject (b'0.00-10') invalid; use 0.0 instead
FloatObject (b'0.00-10') invalid; use 0.0 instead
FloatObject (b'0.00-10') invalid; use 0.0 instead
FloatObject (b'0.00-10') invalid; use 0.0 instead
FloatObject (b'0.00-20') invalid; use 0.0 instead
FloatObject (b'0.00-20') invalid; use 0.0 instead
FloatObject (b'0.00-20') invalid; use 0.0 instead
FloatObject (b'0.00-20') invalid; use 0.0 instead


Filename: S62568_1711141735236001lMUE.pdf, Title: Edge Computing 101
An Introduction to the Smart Edge
March , 24
Chen Su, Sr Technical Product Marketing Manager


In [54]:
import ollama
import re
from jinja2 import Template

In [56]:
print(f"Total {len(texts)} files")

Total 144 files


In [57]:
slide_titles = {}

In [58]:
prompt_template = """
Given the extracted text from the first slide of a keynote (Note that the text can be disorderd):
'Accelerating NetworkX: \nThe Future of Easy Graph Analytics \nMridul Seth - Core Developer, 
NetworkX \nErik Welch - Sr. System Software Engineer, NVIDIA \nRick Ratzel - Sr. System Software 
Engineer, NVIDIA \nNetworkX and backend developers'

Don't use anything word outside the extracted text. Leave anything that does not belong to the title out.
Restore the title of the this keynote in json format is:
{'title': 'Accelerating NetworkX: The Future of Easy Graph Analytics'}

Using the same logic, below is text extracted from the another slide of a keynote:
{{ title_slide }}

The restored title of this keynote in json format is:
"""

for filename, title in texts.items():
    messages = [
        {"role": "system", "content": "You are a helpful agent who help summarize and extract information."},
        {"role": "user", "content": Template(prompt_template).render({'title_slide': title})}
    ]
    response = ollama.chat(model='mistral', messages=messages)

    try: 
        cleaned_title = eval(response['message']['content'])
    except:
        cleaned_title = {"title": ""}
        print(f"Error with {filename}")

    slide_titles[filename] = {
        "title_text": title,
        "cleaned_title": cleaned_title['title']
    }

Error with S62359_1708641953650001ogDb.pdf
Error with S62192_1711057720932001KuIu.pdf
Error with S62493_1710965314889001RDmn.pdf
Error with S62061_EN_Qinginlu_1710870206340001R43v.pdf
Error with S62293 - Entering A New Frontier of AI Networking Innovation_1711049586946001eRPx.pdf
Error with GTC Generative AI Demystified 2024_1711046510606001lWsU.pdf
Error with S62553_1711133464528001x9uD.pdf
Error with S62252_1711140232312001xDPZ.pdf
Error with S62541_1710894498167001OQj7.pdf
Error with EXPT63299_1710972719730001K7Vc.pdf
Error with EXPT62975_1710870460080001QGFy.pdf
Error with S62352_1711151074399001GYsm.pdf
Error with S62529_1710943079626001dMDh.pdf
Error with S62476_1710178444344001l0C2.pdf
Error with S61665_1711140404779001xpnW.pdf
Error with S62149_1711138707003001YSl5.pdf
Error with S62329_1710737882537001hXmm.pdf
Error with S62588_1711137620854001lYOD.pdf
Error with S62549_1710344994751001L71b.pdf
Error with S62739_1710781463997001vNRP.pdf
Error with S62245_1711135121196001aJDF.p

In [113]:
slide_titles

{'S62568_1711141735236001lMUE.pdf': {'title_text': 'Edge Computing 101\nAn Introduction to the Smart Edge\nMarch , 24\nChen Su, Sr Technical Product Marketing Manager',
  'cleaned_title': 'Edge Computing 101: An Introduction to the Smart Edge'},
 'S62359_1708641953650001ogDb.pdf': 'Machine Learning has Taken Weather Forecasting by Storm How About Climate Modeling?',
 'S62192_1711057720932001KuIu.pdf': 'Advanced Performance Optimization in CUDA Igor Terentyev*, NVIDIA DevTech Compute',
 'S62461_1711355892364001HCfs.pdf': {'title_text': 'Exploring Tomorrow’s Industrial AutomationA Journey through the Industrial Metaverse with OmniverseDr. Bahram Torabi, SVP R&DDr. Jan Jarvis, Head of Virtualization SICK AG',
  'cleaned_title': 'Exploring Tomorrow’s Industrial Automation: A Journey through the Industrial Metaverse with Omniverse'},
 'S62981_1711138958547001oCK1.pdf': {'title_text': 'March 18, 2024Unlocking the Power of \nGenAI  for Drug DiscoveryAtlas AI:  \nMapping Data to Knowledge Dani

In [59]:
for k, v in slide_titles.items():
    print(f"{k} || {v['cleaned_title']}")

S62568_1711141735236001lMUE.pdf || Edge Computing 101: An Introduction to the Smart Edge
S62359_1708641953650001ogDb.pdf || 
S62192_1711057720932001KuIu.pdf || 
S62461_1711355892364001HCfs.pdf || Exploring Tomorrow’s Industrial Automation: A Journey through the Industrial Metaverse with Omniverse
S62981_1711138958547001oCK1.pdf || Unlocking the Power of GenAI for Drug Discovery
S63000 - Boosting Business Outcomes with Generative AI (presented by EY)_1710855560264001SG0X.pdf || Boosting business outcomes with generative AI
S62450_1711135639893001FpAD.pdf || March 2024 Frontier AI: in your hands
S61512_1710772888260001wPry.pdf || Can My Model be Hacked? Understanding & Mitigating Vulnerabilities in LLMs - Dr. Peter Garraghan, CEO, Professor
S62493_1710965314889001RDmn.pdf || 
S62061_EN_Qinginlu_1710870206340001R43v.pdf || 
S63058_1710628155053001UUsR.pdf || How Genius Sports Transforms NFL Game Viewing with Accelerated Computing on AWS
S63065_1711133857982001ljf8.pdf || A Blueprint for L

In [108]:
# Manual Review Section
target = 'Lessons from 10 years of building GPU cloud _17107166905980'
for filename, title in texts.items():
    if target in filename:
        print(f'Filename: {filename}, Title: {title}')

Filename: GTC 2024 Draft 02-05-2024 - Lessons from 10 years of building GPU cloud _1710716690598001l0Mb.pdf, Title: 110 Y ear s o f building 
GPU cloud 
Lessons and observations from building a cloud 
infrastructure company at the emergence of the ML / 
AI revolution. 
03 / 19 / 2024 NVIDIA GTC 2024


In [109]:
slide_titles[target] = 'Lessons from 10 years of building GPU cloud'

In [110]:
# Validation

In [115]:
for k, v in slide_titles.items():
    if v['cleaned_title'] == 0:
        print(f"{k} || {v['cleaned_title']}")